In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import plotting

from matplotlib.pyplot import boxplot

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/seoul-bike-rental-ai-pro-iti/sample_submission.csv
/kaggle/input/seoul-bike-rental-ai-pro-iti/train.csv
/kaggle/input/seoul-bike-rental-ai-pro-iti/test.csv


### Exploratory Data Analysis¶


In [2]:
dataset_path = '/kaggle/input/seoul-bike-rental-ai-pro-iti/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
X_test = test_df.drop(columns=['ID'])


print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

The shape of the dataset is (5760, 15).




,ID,Date,y,Hour,Temperature(�C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(�C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5760 entries, 0 to 5759
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         5760 non-null   int64  
 1   Date                       5760 non-null   object 
 2   y                          5760 non-null   int64  
 3   Hour                       5760 non-null   int64  
 4   Temperature(�C)            5760 non-null   float64
 5   Humidity(%)                5760 non-null   int64  
 6   Wind speed (m/s)           5760 non-null   float64
 7   Visibility (10m)           5760 non-null   int64  
 8   Dew point temperature(�C)  5760 non-null   float64
 9   Solar Radiation (MJ/m2)    5760 non-null   float64
 10  Rainfall(mm)               5760 non-null   float64
 11  Snowfall (cm)              5760 non-null   float64
 12  Seasons                    5760 non-null   object 
 13  Holiday                    5760 non-null   objec

In [4]:
df.drop(columns='ID').drop_duplicates()


,Date,y,Hour,Temperature(�C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(�C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,20/11/2018,1317,19,8.7,34,2.2,1857,-6.3,0.0,0.0,0.0,Autumn,No Holiday,Yes
5756,20/11/2018,1078,20,8.5,32,2.6,1789,-7.2,0.0,0.0,0.0,Autumn,No Holiday,Yes
5757,20/11/2018,1020,21,8.8,37,3.8,1750,-5.1,0.0,0.0,0.0,Autumn,No Holiday,Yes
5758,20/11/2018,922,22,8.9,40,3.4,1810,-3.9,0.0,0.0,0.0,Autumn,No Holiday,Yes


In [5]:
df['new_rain'] = 0
df.loc[df['Rainfall(mm)']>0, 'new_rain'] = 1

df['new_snow'] = 0
df.loc[df['Snowfall (cm)']>0, 'new_snow'] = 1


X_test['new_rain'] = 0
X_test.loc[X_test['Rainfall(mm)']>0, 'new_rain'] = 1

X_test['new_snow'] = 0
X_test.loc[X_test['Snowfall (cm)']>0, 'new_snow'] = 1



df['is_night'] = 0
df.loc[(df['Hour'] < 6) | (df['Hour'] > 20), 'is_night'] = 1

X_test['is_night'] = 0
X_test.loc[(X_test['Hour'] < 6) | (X_test['Hour'] > 20), 'is_night'] = 1

In [6]:

df[['Day','Month','Year']] = df.Date.str.split("/",expand=True)
df['Day'] = df['Day'].astype('int64')
df['Year'] = df['Year'].astype('int64')
df['Month'] = df['Month'].astype('int64')

import calendar
df['weekday'] = 0
for ind in df.index:
     df['weekday'][ind]= calendar.weekday(df['Year'][ind], df['Month'][ind] , df['Day'][ind])
        

#df['WEEKDAY'] = pd.to_datetime(df['Date']).dt.weekday_name                 
#df['WEEKNO'] = pd.to_datetime(df['Date']).dt.isocalendar().week

#df['weekend_indi'] = 0          # Initialize the column with default value of 0
#df.loc[df['WEEKDAY'].isin([5, 6]), 'weekend_indi'] = 1


X_test[['Day','Month','Year']] = X_test.Date.str.split("/",expand=True)
X_test['Day'] = X_test['Day'].astype('int64')
X_test['Year'] = X_test['Year'].astype('int64')
X_test['Month'] = X_test['Month'].astype('int64')
#X_test['WEEKNO'] = pd.to_datetime(X_test['Date']).dt.isocalendar().week
#X_test['WEEKDAY'] = pd.to_datetime(X_test['Date']).dt.dayofweek

X_test['weekday'] = 0
for ind in X_test.index:
     X_test['weekday'][ind]= calendar.weekday(X_test['Year'][ind], X_test['Month'][ind] , X_test['Day'][ind])



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
df['not_weekend'] = 0
df.loc[(df['Holiday'] == 'No Holiday') & (df['weekday'] >=5), 'not_weekend'] = 1

X_test['not_weekend'] = 0
X_test.loc[(X_test['Holiday'] == 'No Holiday') & (X_test['weekday'] >=5), 'not_weekend'] = 1

In [8]:
df =  pd.get_dummies(df , columns = ['Seasons'] , drop_first = True)

df = pd.get_dummies(df , columns = ['Holiday'], drop_first = True)

df = pd.get_dummies(df , columns = ['Functioning Day'], drop_first = True)



#df=pd.get_dummies(df , columns=['Month'], prefix='month')












X_test =  pd.get_dummies(X_test , columns = ['Seasons'], drop_first = True)

X_test = pd.get_dummies(X_test, columns = ['Holiday'], drop_first = True)

X_test = pd.get_dummies(X_test, columns = ['Functioning Day'], drop_first = True)



In [9]:
df['rain'] = df.rolling(2, min_periods=1)['Rainfall(mm)'].mean()
df['dryness'] = 1 / (df.rain + 1)

df['snow'] = df.rolling(8, min_periods=1)['Snowfall (cm)'].mean()
df['snowing'] = (df.snow > 0).astype(np.int)

df['invisb'] = df['Visibility (10m)'].max() - df['Visibility (10m)']

df['Wind speed (m/s)'] = np.log(df['Wind speed (m/s)'].astype(float) + 1)
df['Solar Radiation (MJ/m2)'] = np.sqrt(df['Solar Radiation (MJ/m2)'].astype(float))



In [10]:
X_test['rain'] = df.rolling(2, min_periods=1)['Rainfall(mm)'].mean()
X_test['dryness'] = 1 / (X_test.rain + 1)

X_test['snow'] = df.rolling(8, min_periods=1)['Snowfall (cm)'].mean()
X_test['snowing'] = (df.snow > 0).astype(np.int)

X_test['invisb'] = df['Visibility (10m)'].max() - df['Visibility (10m)']

X_test['Wind speed (m/s)'] = np.log(df['Wind speed (m/s)'].astype(float) + 1)
X_test['Solar Radiation (MJ/m2)'] = np.sqrt(df['Solar Radiation (MJ/m2)'].astype(float))

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'y'])
y_train = train_df['y']

X_val = val_df.drop(columns=['ID', 'y'])
y_val = val_df['y']



In [12]:
#using some features (basedup on HeatMap and Corr) to train the classifier 
X_train = X_train [['snowing','dryness','is_night','weekday', 'Temperature(�C)','Wind speed (m/s)' , 'Functioning Day_Yes' ,'not_weekend','Holiday_No Holiday']]
X_val = X_val [['snowing','dryness','is_night','weekday','Temperature(�C)','Wind speed (m/s)'  ,  'Functioning Day_Yes' ,'not_weekend','Holiday_No Holiday']]

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

model_3 = RandomForestRegressor(max_depth = 6, n_estimators=100, random_state=1 , bootstrap = True)


model_3.fit(X_train , y_train)
pre = model_3.predict(X_val)


print(mean_squared_log_error(y_val , pre))




0.508667116817565


In [14]:
# You should update/remove the next line once you change the features used for training

X_test = X_test [['snowing','dryness','is_night','weekday','Temperature(�C)','Wind speed (m/s)'  ,  'Functioning Day_Yes' ,'not_weekend','Holiday_No Holiday']]
y_test_predicted = model_3.predict(X_test)

test_df['y'] = y_test_predicted.astype(int)

test_df.head()

,ID,Date,Hour,Temperature(�C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(�C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,y
0,5760,21/12/2017,0,-1.3,90,2.0,259,-2.7,0.0,0.0,2.2,Winter,No Holiday,Yes,183
1,5761,21/12/2017,1,-1.8,90,1.6,263,-3.2,0.0,0.0,2.2,Winter,No Holiday,Yes,177
2,5762,21/12/2017,2,-2.2,88,1.4,348,-3.9,0.0,0.0,2.2,Winter,No Holiday,Yes,179
3,5763,21/12/2017,3,-2.9,89,1.3,288,-4.4,0.0,0.0,2.2,Winter,No Holiday,Yes,178
4,5764,21/12/2017,4,-3.6,91,1.2,241,-4.8,0.0,0.0,2.2,Winter,No Holiday,Yes,183


In [15]:
test_df[['ID', 'y']].to_csv('/kaggle/working/submission.csv', index=False)
